# Julia Review (ECON627 UBC)

****

If we choose to store our packages (and their versions), we want to activate our Project.toml file. 

In [1]:
using Pkg
Pkg.activate(joinpath(pwd(),"..")) ;

  Activating project at `c:\Users\steve\Documents\GitHub\ECON627_UBC.jl`


****

In this course you learned that there is a large class of estimators that, under suitable regularity conditions, admit an asymptotic linear representation: 

\begin{align*}
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) &= - \left[ \frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} \right]^{-1} \sqrt n \frac{\partial Q_n(\theta_0)}{\partial \theta} + o_P(1)  \\
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) &= \frac{1}{\sqrt n} \sum_{i=1}^n \xi_i + o_P(1) \quad , 
\end{align*}

where $\xi_i$ is known as the influence function, which are i.i.d and centered at zero $\mathbf{E}[\xi_i]=0$. By the CLT of i.i.d observations we have that 

\begin{equation}
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) \rightarrow_d \mathcal{N}( 0, \mathbf{E}[\xi_i \xi_i'])
\end{equation}



**************
#### How to know if I'm missing an extra "n" when I compute the standard errors?
Just follow these steps:

- Compute analytically the asymptotic variance.
- Replace every expectation by sample average (every sample average must be divided by n!).
- The standard errors will be ``sqrt.(diag(avar))/sqrt(n)``


***
Before we do anything, we set a random seed to ensure replicability of our code.

In [2]:
using Random
Random.seed!(1234);

Load the ECON627 package 

In [3]:
using ECON627_UBC

## 1. Least Squares 

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ (Y_i - X_i'\theta)^2 \right]/2$
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n} \sum_{i=1}^n (Y_i - X_i'\theta)^2/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \frac{1}{n} \sum_{i=1}^n (Y_i-X_i'\theta_0)X_i $
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n X_i X_i' \rightarrow_p \mathbf{E} X_i X_i'$
 - Influence function: $\xi_i :=   (\mathbf{E}X_i X_i')^{-1} X_i U_i$ 
 
Therefore, the asymptotic variance is given by 
$ (\mathbf{E}X_i X_i')^{-1} \mathbf{E}[ U_i^2 X_i X_i'] (\mathbf{E}X_i X_i')^{-1}  $

In [4]:
using Parameters, LinearAlgebra, Distributions

In [5]:
function data_generating_ols(;n=1000)
    #Define the Multivariate Normal Distribution instance as an example
    mvnormal = MvNormal([1.0; 2.0], [1 0.7; 0.7 1])
    
    #Matrix X (n x 2 ) and ε (n x 1)
    X = rand(mvnormal,n)
    X = transpose(X)
    ε = randn(n, 1)
    
    #Vector y
    y = X*[1.0;1.0] + ε
        
    #Return the data
    return (X = X , y = y)
end    

data_generating_ols (generic function with 1 method)

In [6]:
@unpack X,y = data_generating_ols(n=1000)
@unpack θhat, se = ols(X,y)

(θhat = [0.9563864710176802; 1.0323477434590973;;], se = [1.3591890782320168, 0.8250537779697441])

In [7]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack X,y = data_generating_ols(;n=n)
    @unpack θhat, se = ols(X,y)
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    
    #Coverage of θ[1]
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.955

## 2. Linear GMM

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ Z_i (Y_i - X_i'\theta) \right]' A'A \mathbf{E} \left[ Z_i (Y_i - X_i'\theta) \right]/2$ 
 
 Notice that if there's correct specification, $\mathbf{E}  \left[ Z_i (Y_i - X_i'\theta) \right] = 0$ ,this population criterion has a minimum value of 0 for any choice of norm $A$.
 
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n}  \left[ \sum_{i=1}^n Z_i (Y_i - X_i'\theta) \right]' A_n'A_n  \left[ \sum_{i=1}^n Z_i (Y_i - X_i'\theta) \right]/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta'} = - \frac{1}{n} \sum_{i=1}^n X_i Z_i' A_n' A_n \frac{1}{n} \sum_{i=1}^n Z_i(Y_i-X_i'\theta_0)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n X_i Z_i' A_n'A_n  \frac{1}{n} \sum_{i=1}^n Z_i X_i' \rightarrow_p \Gamma_0 A'A \Gamma_0$
 - Influence function: $\xi_i :=   (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A Z_i U_i$ 
 
Therefore, the asymptotic variance is given by 
$ (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A \mathbf{E}[ U_i^2 Z_i Z_i'] A'A \Gamma_0 (\Gamma_0' A'A \Gamma_0)^{-1}  $

In [8]:
using Parameters, LinearAlgebra, Distributions

In [9]:
function data_generating_gmm(;n=1000)
   
    ϵ=randn(n,1)
    V=randn(n,1)
    U=0.5*ϵ+V
    
    Z1=randn(n,1)
    Xexo=randn(n,3)
    
    Xendo=Z1+Xexo*ones(3,1)+V
    Y=Xendo+Xexo*ones(3,1)+U

    return (Y=Y, Xendo = Xendo, Xexo=Xexo, Z1 = Z1)  
end    

data_generating_gmm (generic function with 1 method)

In [10]:
@unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(n=1000)
X=hcat(Xendo,Xexo)
Z=hcat(Xexo,Z1)
@unpack θhat, se = TSLS(Y,X,Z)

(θhat = [0.9938392308323587; 0.9909012899063977; 0.9785738392953629; 0.9930816515690302;;], se = [1.1176338684691618, 1.5939775190723693, 1.5579986598162425, 1.5706964438137785])

In [11]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    @unpack θhat, se = TSLS(Y,X,Z)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.943

I'll also show results for GMM with $W=I$

In [12]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    W = Matrix(I,size(Z,2),size(Z,2))
    @unpack θhat, se = GMM(W,Y,X,Z)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.956

And two-step efficient GMM 

In [13]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    @unpack θhat, se = TSGMM(Y,X,Z)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.945

## 3. Non-linear GMM

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[g(W_i,\theta) \right]' A'A \mathbf{E} \left[ g(W_i,\theta) \right]/2$ 
 
 Notice that if there's correct specification, $\mathbf{E}  \left[ g(W_i,\theta)\right] = 0$ ,this population criterion has a minimum value of 0 for any choice of norm $A$.
 
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n}  \left[ \sum_{i=1}^n g(W_i,\theta) \right]' A_n'A_n  \left[ \sum_{i=1}^n g(W_i,\theta) \right]/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} =  \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)'}{\partial \theta}\right) A_n'A_n' \left( \frac{1}{n} \sum_{i=1}^n  g(W_i,\theta_0)\right)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)}{\partial \theta}\right) A_n'A_n' \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)}{\partial \theta'}\right) + \left[I_k \otimes \left( \frac{1}{n} \sum_{i=1}^n  g(W_i,\theta_0)\right) A_n'A_n\right] \left[ \frac{1}{n} \sum_{i=1}^n \frac{\partial}{\partial \theta'} vec \left( \frac{\partial g(W_i,\theta_0)}{\partial \theta'} \right) \right]$
 $\rightarrow_p \Gamma_0' A'A\Gamma_0 + \left[I_k \otimes \mathbf{E}[g(W_i,\theta_0)'A'A] \right] \mathbf{E} \frac{\partial}{\partial \theta'}vec \left( \frac{\partial g(W_i,\theta_0)}{\partial \theta'} \right)  $
 
 where the second term is 0 if correctly specified. In the influence function below I'll assume it's zero to avoid writing too much algebra.
 
 - Influence function: $\xi_i :=   (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A g(W_i,\theta_0)$ 
 
Therefore, the asymptotic variance is given by 
$ (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A \mathbf{E}[ g(W_i,\theta_0) g(W_i,\theta_0)'] A'A \Gamma_0 (\Gamma_0' A'A \Gamma_0)^{-1}  $

In [14]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [15]:
function data_generating_gmm(;n=1000)
   
    ϵ=randn(n,1)
    V=randn(n,1)
    U=0.5*ϵ+V

    #2 instruments for 1 endogenous : overidentified
    Z1=randn(n,2)
    Xexo=randn(n,3)
    
    Xendo=Z1*ones(2,1)+Xexo*ones(3,1)+V
    Y=Xendo+Xexo*ones(3,1)+U

    return (Y=Y, Xendo = Xendo, Xexo=Xexo, Z1 = Z1)  
end    

data_generating_gmm (generic function with 1 method)

In [16]:
@unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(n=1000)
X=hcat(Xendo,Xexo)
Z=hcat(Xexo,Z1)
W = inv(Z'*Z)
g = (y,x,z,θ) -> z*(y-x*θ)

#24 (generic function with 1 method)

In [17]:
@unpack θhat, se = NLGMM(Y, X, Z,W,g) 

MethodError: MethodError: no method matching (::ECON627_UBC.var"#10#21"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#gvec#18"{ECON627_UBC.var"#7#17"{var"#24#25", Int64}, Int64}})(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{ECON627_UBC.var"#11#22"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#10#21"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#gvec#18"{ECON627_UBC.var"#7#17"{var"#24#25", Int64}, Int64}}}, Float64}, Float64, 1}}, ::Matrix{Float64}, ::Matrix{Float64}, ::Matrix{Float64}, ::Matrix{Float64})
Closest candidates are:
  (::ECON627_UBC.var"#10#21")(::Any) at c:\Users\steve\Documents\GitHub\ECON627_UBC.jl\src\ECON627_UBC.jl:149

In [18]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    @unpack θhat, se = NLGMM(Y, X, Z,W,g) 
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

TaskFailedException: TaskFailedException

    nested task error: MethodError: no method matching (::ECON627_UBC.var"#10#21"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#gvec#18"{ECON627_UBC.var"#7#17"{var"#24#25", Int64}, Int64}})(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{ECON627_UBC.var"#11#22"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#10#21"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#gvec#18"{ECON627_UBC.var"#7#17"{var"#24#25", Int64}, Int64}}}, Float64}, Float64, 1}}, ::Matrix{Float64}, ::Matrix{Float64}, ::Matrix{Float64}, ::Matrix{Float64})
    Closest candidates are:
      (::ECON627_UBC.var"#10#21")(::Any) at c:\Users\steve\Documents\GitHub\ECON627_UBC.jl\src\ECON627_UBC.jl:149
    Stacktrace:
      [1] (::ECON627_UBC.var"#11#22"{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, Matrix{Float64}, ECON627_UBC.var"#10#21"{Matrix{Float64}, Matrix{Float64}, Ma

## 4. MLE

 - Population Criterion : $Q(\theta) = -\mathbf{E} \log f(W_i,\theta)$ 
 - Sample Criterion: $Q_n(\theta) = -\frac{1}{n}   \sum_{i=1}^n f(W_i,\theta) $
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = -\frac{1}{n}   \sum_{i=1}^n \frac{\partial}{\partial \theta} f(W_i,\theta_0)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = -\frac{1}{n}   \sum_{i=1}^n \frac{\partial^2}{\partial \theta \partial \theta'} f(W_i,\theta)$
 $\rightarrow_p -\mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  $ 
 - Influence function: $\xi_i := \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} \frac{\partial}{\partial \theta} \log f(W_i,\theta_0)  $ 
 
Therefore, the asymptotic variance is given by 
$ \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} \mathbf{E}[ \frac{\partial}{\partial \theta} \log f(W_i,\theta_0) \frac{\partial}{\partial \theta'} \log f(W_i,\theta_0) ] \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} $

Under correctly specified MLE the matrix inside $\Omega_0$ equals the Hessian $B_0$, and the asymptotic variance is reduced to the inverse of the Hessian matrix. If that is not the case, the minimizer is a pseudo-true parameter and this is called the Quasi-MLE approach.  

In [19]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [20]:
function data_generating_mle(;n=1000)
      
    X=randn(n)
    u=randn(n)
    Y=(u .< 1.0 .+ X)
    X = hcat(ones(n), X)
    return (Y=Y,X= X )
end

Φ(v)=cdf(Normal(0,1),v)
ϕ(v)=pdf(Normal(0,1),v)
Λ(v) = 1 / (1 + exp(-v))



Λ (generic function with 1 method)

In [21]:
function LogL_Probit(θ,X,Y)
    n=length(Y)
    LL=0.0
    X1 = X[:,1] 
    X2 = X[:,2]
    for i=1:n
        indx=X1[i]*θ[1]+X2[i]*θ[2]
        LL -=Y[i]*log(Φ(indx))+(1-Y[i])*log(1-Φ(indx))
    end
    return LL/n
end


function LogL_Logit(θ,X,Y)
    n=length(Y)
    LL=0.0
    X1 = X[:,1] 
    X2 = X[:,2]
    for i=1:n
        indx=X1[i]*θ[1]+X2[i]*θ[2]
        LL -=Y[i]*log(Λ(indx))+(1-Y[i])*log(1-Λ(indx))
    end
    return LL/n
end

LogL_Logit (generic function with 1 method)

In [ ]:
@unpack X,Y = data_generating_mle(;n=1000)
@unpack θhat_probit, se_probit = MLE(Y,X,LogL_Probit)
@unpack θhat_logit, se_logit = MLE(Y,X,LogL_Logit)


In [ ]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack X,Y = data_generating_mle(;n=1000)
    @unpack θhat, se = MLE(X,Y)

    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

## 5. Non-linear Least Squares

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ (Y_i - g(X_i,\theta))^2 \right]/2$
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n} \sum_{i=1}^n (Y_i - g(X_i,\theta))^2/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \frac{1}{n} \sum_{i=1}^n \frac{\partial}{\partial \theta} g(X_i,\theta_0) (Y_i-g(X_i,\theta_0))$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n \{ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)' - U_i \frac{\partial^2}{\partial \theta \partial \theta'} g(X_i,\theta_0) \}  \rightarrow_p \mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right] - \mathbf{E} \left[ U_i \frac{\partial^2}{\partial \theta \partial \theta'} g(X_i,\theta_0)\right]$
 
 where the second component is equal to 0 if the model is correctly specified $\mathbf{E}[U_i \mid X_i] =0$ and the law of iterated expectations. I will assume the model is correctly specified to simplify the algebra below. 
 
 - Influence function: $\xi_i :=   (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \frac{\partial}{\partial \theta} g(X_i,\theta_0) U_i$ 
 
Therefore, the asymptotic variance is given by 
$  (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \mathbf{E}[ U_i^2 \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)']  (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1}  $

In [24]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [25]:
function data_generating_nls(;n=1000)
    #Define the Multivariate Normal Distribution instance
    mvnormal = MvNormal([1.0; 2.0], [1 0.7; 0.7 1]);
    
    #Define Logistic Link function 
    g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1
    
    #Matrix X (n x 2 ) and ε (n x 1)
    X = rand(mvnormal,n)
    X = transpose(X)
    ε = randn(n, 1)
    
    #Vector y
    Y = g(X,[1.0, -1.0]) + ε
        
    #Return the data
    return (X = X , Y = Y)
end

data_generating_nls (generic function with 1 method)

In [26]:
@unpack X,Y = data_generating_nls()
g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1

@unpack θhat, se = nls(g,Y,X)

(θhat = [0.8376579661664639, -0.7386012310042075], se = [0.22300703875034458, 0.16026591368123383])

In [27]:
n=1000
R=10^3

θcoverage95 = zeros(R)
g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1

Threads.@threads for r=1:R
    
    @unpack X,Y = data_generating_nls(;n=n)
    @unpack θhat, se = nls(g,Y,X)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.044

## 6. Minimum Distance

 - Population Criterion : $Q(\theta) = (\pi_0 - g(\theta))' A'A (\pi_0 - g(\theta))/2$, 
 - Sample Criterion: $Q_n(\theta) = (\hat{\pi}_n - g(\theta))' A_n'A_n (\hat{\pi}_n - g(\theta))/2$
 
  where $\sqrt n (\hat{\pi}_n - \pi_0 ) \rightarrow_p \mathcal{N}(0,V_0)$ is a first-step estimator.

 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \left( \frac{\partial g(\theta_0)}{\partial \theta} \right)' A_n'A_n (\hat{\pi}_n - g(\theta_0))$
  - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \left( \frac{\partial g(\theta_0)}{\partial \theta} \right)' A_n'A_n  \left( \frac{\partial g(\theta_0)}{\partial \theta} \right) + \left[I_k \otimes (\hat{\pi}_n-g(\theta_0))' A_n'A_n \right] \frac{\partial}{\partial \theta'} vec\left(  \frac{\partial g(\theta_0)}{\partial \theta'}\right) $
$\rightarrow_p \Gamma_0 A'A\Gamma_0 + \left[ I_k \otimes (\pi_0-g(\theta_0))' A'A  \right] \frac{\partial}{\partial \theta'} vec\left(  \frac{\partial g(\theta_0)}{\partial \theta'}\right) $
 
 where the second component is equal to 0 if the model is correctly specified $\pi_0 = g(\theta_0)$. I will assume the model is correctly specified to simplify the algebra below. 
 
 - Influence function: $\xi_i :=   (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \frac{\partial}{\partial \theta} g(X_i,\theta_0) U_i$ 
 
Therefore, the asymptotic variance is given by 
$  ( \Gamma_0 A'A \Gamma_0 )^{-1} \Gamma_0 A'A \xi_i^{\pi} $, 
where $\xi_i^{\pi}$ is the influence function that comes from $\sqrt n (\hat{\pi}_n - \pi_0 )$ (i.e. the first step).


## 7. Quantile Regression 

Recall that quantile regression is a special case where we encounter differentiability issues. The approach taken here is to construct a stochastic equicontinuous process such that we obtain a smooth function where we can apply a mean value expansion around the true value $\theta_{\tau,0}$.

This approach yields the following asymptotic linear representation

\begin{align*}
\sqrt n (\hat{\theta}_{\tau,n} - \theta_{\tau,0} ) &= - \left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1} \frac{1}{\sqrt n} \sum_{i=1}^n \{  (\tau - \mathbf{1}\{ Y_i < X_i'\theta_{\tau,0} \} X_i \} + o_P(1)
\end{align*}

This implies that the influence function is given by 
$\xi_i = \left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1} \{  (\tau - \mathbf{1}\{ Y_i < X_i'\theta_{\tau,0} \} X_i \}$, and the asymptotic variance is given by $\left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1}\mathbf{E}[\tau (1-\tau) X_i X_i'] \left(    \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1}$ by the L.I.E.



In [28]:
using LinearAlgebra, JuMP, HiGHS, Optim, Statistics,Plots, Parameters

In [29]:
function data_generating_qr(;n=1000)
    X=randn(n,1).^2
    U=rand(n,1)
    Y=X .* (U .-0.5)
    return (Y=Y, X=X)
end

#If you wanna estimate asy-var
K(v)= abs(v) - 1 < 0 ?   0.5* (abs(v) - 1) : 0.0;

In [30]:
function estimate_qreg(;Y,X,τ)
    
    QR=Model(HiGHS.Optimizer)
    set_silent(QR)
    n=length(Y)
    
    c=[τ*ones(n,1);(1-τ)*ones(n,1);];
    @variable(QR,x[1:2*n]>=0.0); #this includes the non-negativity constraint on z^{+} and z^{-}
    @variable(QR,-100.0<=b<=100.0); #if you want to estimate vector b you use @variable(QR,-100.0<=b[1:k]<=100.0) where k is dimension
    @objective(QR,Min,sum(c[i]*x[i] for i in 1:(2*n)));
    @constraint(QR,constraint[i in 1:n],x[i]-x[n+i]+sum(X[i,:].*b)==Y[i]); #the equality constraint
    optimize!(QR);
    estr=value.(b)
    
    
    #Drop this if you don't care about asy-var
    "Calculation of Asymptotic Variance"
    n=length(Y)
    rhat= Y - X * estr;
    h = 1.06 * std(rhat) * n ^ (-1 / 5);
    Khat =  K.(rhat./h) ;

    Ωhat = τ * (1 - τ) * (X' * X);
    Bhat = ((X.*repeat(Khat,1,size(X,2)))' * X) ./ h;
    v = n*(Bhat \ Ωhat / Bhat);
    
    
    return (θhat = estr , se = sqrt.(diag(v)) )
end


estimate_qreg (generic function with 1 method)

In [31]:
@unpack Y, X =data_generating_qr();
@unpack θhat, se=estimate_qreg(;Y=Y,X=X,τ=0.25)

In [ ]:
n=1000
R=10^3
τcheck=0.75
θτ=τcheck-0.5
θcoverage95 = zeros(R)

# You can uncomment but it does take time to run, and you probably won't need this :) 
#Threads.@threads for r=1:R
    
    #@unpack Y, X =data_generating_qr(;n=n);
    #@unpack θhat, se=estimate_qreg(;Y=Y,X=X,τ=τcheck)
    
    #lower_ci  = θhat .- 1.96.*se/sqrt(n)
    #upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    #θcoverage95[r] = (lower_ci[1]<θτ)*(upper_ci[1]>θτ)
#end

sum(θcoverage95)/R